In [123]:
import pandas as pd
import numpy as np

import warnings

from google.cloud import bigquery
from google.cloud import bigquery_storage
creds = '/Users/rokokkula/Documents/cprtpr-datastewards-sp1-stewardapp-prbq_2_1.json'
client = bigquery.Client.from_service_account_json(json_credentials_path=creds)
bqstorageclient = bigquery_storage.BigQueryReadClient(credentials=creds)
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [127]:
import pandas as pd

# Define the path to the Excel file
file_path = '/Users/rokokkula/Downloads/Compiled Sales - 2024 - Company B.xlsx'

# Read the specific sheet "Q4-2024"
sheet_name = 'Q4-2024'
# Load the Excel file into a DataFrame
df = pd.read_excel(file_path, sheet_name=sheet_name)


In [128]:
df.head()

,Sale Date,Sale Week,Sale Month,Sale Quarter,Source,#,Year,Make,Model,VIN,...,State,Damage,Damage Type,Estimate,Odometer,Odom Status,Run / Drive,Title,Title Type,itemDisplayStatus
0,2024-12-27,52,12,4,B,0,2020,FORD,F-150 XL,1FTEX1EP4LF******,...,NS,Front,UK,18443.50,169360.0,Actual,No,- NS-NOT BRANDED,NaN,Sold
1,2024-12-27,52,12,4,B,0,2022,MAZDA,CX-9,JM3TCBCY9N0******,...,NS,Theft,VI,3820.00,23528.0,Actual,Yes,- NS-NOT BRANDED,NaN,Sold
2,2024-12-27,52,12,4,B,0,2007,BMW,5 series,WBANF33537C******,...,NS,Front,UK,713.00,0.0,NaN,No,- NB-NOT BRANDED,NaN,Sold
3,2024-12-27,52,12,4,B,0,2020,BUICK,ENCORE GX,KL4MMGSL3LB******,...,ON,Front,UK,12037.13,72061.0,Actual,No,- ON-NOT BRANDED,NaN,If Bid
4,2024-12-27,52,12,4,B,0,2011,NISSAN,VERSA 1.8S,3N1BC1CP1BL******,...,ON,Front,UK,3569.82,191589.0,Actual,No,- ON-NOT BRANDED,NaN,Sold


In [140]:
query = """
select yard_name,count(*)
from `cprtpr-dataplatform-sp1`.usviews.v_us_lot_fact
where inv_dt between ('2024-10-01') and ('2024-12-31')
AND lot_type_cd IN ('V')
AND yard_country_cd = 'USA'
and yard_state_cd = 'AL'
and seller_type='I'
group by 1
"""
data = client.query(query).to_dataframe()

In [141]:
data
df.loc[df['State']=='AL ',]['Yard'].value_counts()

,yard_name,f0_
0,AL - TANNER,4120
1,AL - BIRMINGHAM,5357
2,AL - MOBILE,2605
3,AL - DOTHAN,1550
4,AL - MONTGOMERY,2947
5,AL - MOBILE SOUTH,1636


Yard
Birmingham    3590
Huntsville    1572
Dothan        1015
Name: count, dtype: int64

In [9]:
data['yard_id']

KeyError: 'yard_id'

In [12]:
data.yard_name

0    AL - BIRMINGHAM
Name: yard_name, dtype: object

In [96]:
df.head()

,lot_nbr,vin_nbr,yard_nbr,lot_year,make,lotModel,groupModel,ACV,plugACV,lot_cat_long_desc,...,rearLeftBumperDamage,rearLeftWindshieldDamage,rearLeftWheelDamage,rearLeftTrunkDamage,airbagsDeployedDetected,suspensionOrAxleDetected,dislocationDetected,flatTireDetected,lids_version,teardown_flag
0,40267394,JF1VBAA62P9813294,7,2023,SUBA,WRX,WRX,27320.0,27875.0,A - AUTOMOBILE,...,None,None,0.0,None,0,0,0,0,"""4.0.0""",No
1,40207774,JA4AZ3A39LZ018351,116,2020,MITS,OUTLANDER,OUTLANDER,18125.0,18575.0,S - SUV'S,...,0.0,0.0,0.0,None,0,0,0,0,"""4.0.0""",No
3,39430294,5FPYK3F88NB006897,114,2022,HOND,RIDGELINE,RIDGELINE,32103.0,38175.0,P - PICKUP'S,...,0.0,None,0.0,0.0,0,0,0,0,"""4.0.0""",No
4,39295194,KNDPNCAC3J7428616,27,2018,KIA,SPORTAGE E,SPORTAGE,19000.0,17475.0,V - VANS/MINIVANS,...,0.0,0.0,0.0,0.0,1,1,0,1,"""4.0.0""",No
6,39684744,4S3GTAB66M3706553,85,2021,SUBA,IMPREZA,IMPREZA,15967.5,21000.0,A - AUTOMOBILE,...,0.3291252804,0.0361602982,0.0,0.8941281139,1,0,1,0,"""4.0.0""",No


In [18]:
df['Sale Date'].min()

Timestamp('2024-10-01 00:00:00')

In [21]:
df['Sale Date'].max()

Timestamp('2024-12-27 00:00:00')

Yard
Lexington     5552
Greenville    1559
Charleston    1398
Name: count, dtype: int64

In [97]:
df['State'].unique()

KeyError: 'State'

In [111]:
query = """SELECT
    lot_nbr,
    vin_nbr,
    yard_nbr,
    lot_year,
    make,
    lotModel,
    groupModel,
    lotACV as ACV,
    plugACV,
    lot_cat_long_desc,
    damage_type_cd,
    repairCost,
    inv_dt as invDt,
    proquote_amt,
    seller_parent_company,
    total_adv_chrgs_amt,
    total_adv_chrgs_amt/high_bid_amt as totalAdvChrgsPercent,
    repairCost/lotACV as repair_percent,
    high_bid_amt as highBidAmt,
    -- output_details_json,
    JSON_EXTRACT(output_details_json, '$.fr.image_url') AS frontRightImageUrl,
    JSON_EXTRACT(output_details_json, '$.fl.image_url') AS frontLeftImageUrl,
    JSON_EXTRACT(output_details_json, '$.rr.image_url') AS rearRightImageUrl,
    JSON_EXTRACT(output_details_json, '$.rl.image_url') AS rearLeftImageUrl,
    JSON_EXTRACT(output_details_json, '$.fr.ext_dmg_scores.f_bmpr') AS frontRightBumperDamage,
    JSON_EXTRACT(output_details_json, '$.fr.ext_dmg_scores.f_dr') AS frontRightDoorDamage,
    JSON_EXTRACT(output_details_json, '$.fr.ext_dmg_scores.f_fndr') AS frontRightFenderDamage,
    JSON_EXTRACT(output_details_json, '$.fr.ext_dmg_scores.hdlght') AS frontRightHeadlightDamage,
    JSON_EXTRACT(output_details_json, '$.fr.ext_dmg_scores.hood') AS frontRightHoodDamage,
    JSON_EXTRACT(output_details_json, '$.fr.ext_dmg_scores.f_wndshld') AS frontRightWindshieldDamage,
    JSON_EXTRACT(output_details_json, '$.fr.ext_dmg_scores.whl') AS frontRightWheelDamage,
    JSON_EXTRACT(output_details_json, '$.fl.ext_dmg_scores.f_bmpr') AS frontLeftBumperDamage,
    JSON_EXTRACT(output_details_json, '$.fl.ext_dmg_scores.f_dr') AS frontLeftDoorDamage,
    JSON_EXTRACT(output_details_json, '$.fl.ext_dmg_scores.f_fndr') AS frontLeftFenderDamage,
    JSON_EXTRACT(output_details_json, '$.fl.ext_dmg_scores.hdlght') AS frontLeftHeadlightDamage,
    JSON_EXTRACT(output_details_json, '$.fl.ext_dmg_scores.hood') AS frontLeftHoodDamage,
    JSON_EXTRACT(output_details_json, '$.fl.ext_dmg_scores.f_wndshld') AS frontLeftWindshieldDamage,
    JSON_EXTRACT(output_details_json, '$.fl.ext_dmg_scores.whl') AS frontLeftWheelDamage,
    JSON_EXTRACT(output_details_json, '$.rr.ext_dmg_scores.r_bmpr') AS rearRightBumperDamage,
    JSON_EXTRACT(output_details_json, '$.rr.ext_dmg_scores.r_dr') AS rearRightDoorDamage,
    JSON_EXTRACT(output_details_json, '$.rr.ext_dmg_scores.r_qrtr_pnl') AS rearRightQuarterPanelDamage,
    JSON_EXTRACT(output_details_json, '$.rr.ext_dmg_scores.tllght') AS rearRightTaillightDamage,
    JSON_EXTRACT(output_details_json, '$.rr.ext_dmg_scores.r_wndshld') AS rearRightWindshieldDamage,
    JSON_EXTRACT(output_details_json, '$.rr.ext_dmg_scores.whl') AS rearRightWheelDamage,
    JSON_EXTRACT(output_details_json, '$.rr.ext_dmg_scores.trnk') AS rearRightTrunkDamage,
    JSON_EXTRACT(output_details_json, '$.rl.ext_dmg_scores.r_dr') AS rearLeftDoorDamage,
    JSON_EXTRACT(output_details_json, '$.rl.ext_dmg_scores.r_qrtr_pnl') AS rearLeftQuarterPanelDamage,
    JSON_EXTRACT(output_details_json, '$.rl.ext_dmg_scores.tllght') AS rearLeftTaillightDamage,
    JSON_EXTRACT(output_details_json, '$.rl.ext_dmg_scores.r_bmpr') AS rearLeftBumperDamage,
    JSON_EXTRACT(output_details_json, '$.rl.ext_dmg_scores.r_wndshld') AS rearLeftWindshieldDamage,
    JSON_EXTRACT(output_details_json, '$.rl.ext_dmg_scores.whl') AS rearLeftWheelDamage,
    JSON_EXTRACT(output_details_json, '$.rl.ext_dmg_scores.trnk') AS rearLeftTrunkDamage,
    GREATEST(
        JSON_EXTRACT(output_details_json, '$.fr.ext_dmg_scores.arbg_vsbl'),
        JSON_EXTRACT(output_details_json, '$.fl.ext_dmg_scores.arbg_vsbl'),
        JSON_EXTRACT(output_details_json, '$.rr.ext_dmg_scores.arbg_vsbl'),
        JSON_EXTRACT(output_details_json, '$.rl.ext_dmg_scores.arbg_vsbl')
    ) AS airbagsDeployedDetected,
    GREATEST(
        JSON_EXTRACT(output_details_json, '$.fr.ext_dmg_scores.spnsn_axl_dmg'),
        JSON_EXTRACT(output_details_json, '$.fl.ext_dmg_scores.spnsn_axl_dmg'),
        JSON_EXTRACT(output_details_json, '$.rr.ext_dmg_scores.spnsn_axl_dmg'),
        JSON_EXTRACT(output_details_json, '$.rl.ext_dmg_scores.spnsn_axl_dmg')
    ) AS suspensionOrAxleDetected,
    GREATEST(
        JSON_EXTRACT(output_details_json, '$.fr.ext_dmg_scores.dslctn'),
        JSON_EXTRACT(output_details_json, '$.fl.ext_dmg_scores.dslctn'),
        JSON_EXTRACT(output_details_json, '$.rr.ext_dmg_scores.dslctn'),
        JSON_EXTRACT(output_details_json, '$.rl.ext_dmg_scores.dslctn')
    ) AS dislocationDetected,
    GREATEST(
        JSON_EXTRACT(output_details_json, '$.fr.ext_dmg_scores.flt_tr'),
        JSON_EXTRACT(output_details_json, '$.fl.ext_dmg_scores.flt_tr'),
        JSON_EXTRACT(output_details_json, '$.rr.ext_dmg_scores.flt_tr'),
        JSON_EXTRACT(output_details_json, '$.rl.ext_dmg_scores.flt_tr')
    ) AS flatTireDetected,
    JSON_EXTRACT(output_details_json, '$.lids_version') AS lids_version
FROM `cprtpr-dataplatform-sp1`.usviews.v_us_lot_image_damage_scores_fact a1
JOIN (
    SELECT
        lot_nbr,
        yard_nbr,
        vin_nbr,
        lot_year,
        lot_cat_long_desc,
        damage_type_cd,
        NULLIF(TRIM(lot_make_cd), '') AS make,
        NULLIF(TRIM(lot_model), '') AS lotModel,
        NULLIF(TRIM(grp_model), '') AS groupModel,
        NULLIF(acv, 0) AS lotACV,
        NULLIF(plug_lot_acv, 0) AS plugACV,
        NULLIF(repair_cost, 0) AS repairCost,
        proquote_amt,
        inv_dt,
        seller_parent_company,
        total_adv_chrgs_amt,
        high_bid_amt
    FROM `cprtpr-dataplatform-sp1`.usviews.v_us_lot_fact
    WHERE
        inv_dt between '2024-01-01' and '2025-01-01'
        AND lot_type_cd IN ('V')
        AND yard_country_cd = 'USA'
        AND ACV > 1000
        and high_bid_amt>100
        and repair_cost>1000
      and seller_type='I'
      and seller_parent_company='PRG'
and (acv-repair_cost)*0.5<=high_bid_amt
and repair_cost/acv>0.8
    and total_adv_chrgs_amt/high_bid_amt>0.25
    and lot_year>2017
and acv!=repair_cost


) lots ON a1.lot = lots.lot_nbr
WHERE a1.lids_version IN ('3.0.1', '3.0.2')
limit 10000"""
df = client.query(query).to_dataframe()

df.shape

(2503, 56)

In [ ]:
export PATH="$PATH:/Users/rokokkula/Downloads/jdk-24.jdk/Contents/Home/bin”

In [143]:
query = """SELECT * from `cprtpr-dataplatform-sp1`.usviews.v_us_bids_fact a1
limit 1000
"""
df = client.query(query).to_dataframe()

df.shape

(1000, 23)

In [115]:
# Define the columns to convert to float
columns_to_convert = [
    'frontRightBumperDamage', 'frontRightDoorDamage', 'frontRightFenderDamage', 'frontRightHeadlightDamage',
    'frontRightHoodDamage', 'frontRightWindshieldDamage', 'rearRightBumperDamage', 'rearRightDoorDamage',
    'rearRightQuarterPanelDamage', 'rearRightTaillightDamage', 'rearRightTrunkDamage', 'rearRightWheelDamage',
    'frontLeftBumperDamage', 'frontLeftDoorDamage', 'frontLeftFenderDamage', 'frontLeftHeadlightDamage',
    'frontLeftHoodDamage', 'frontLeftWindshieldDamage', 'rearLeftBumperDamage', 'rearLeftDoorDamage',
    'rearLeftQuarterPanelDamage', 'rearLeftTaillightDamage', 'rearLeftTrunkDamage', 'rearLeftWheelDamage'
]

# Convert the specified columns to float
for col in columns_to_convert:
    df[col] = df[col].astype(float)

# Verify the changes
df[columns_to_convert].dtypes


frontRightBumperDamage         float64
frontRightDoorDamage           float64
frontRightFenderDamage         float64
frontRightHeadlightDamage      float64
frontRightHoodDamage           float64
frontRightWindshieldDamage     float64
rearRightBumperDamage          float64
rearRightDoorDamage            float64
rearRightQuarterPanelDamage    float64
rearRightTaillightDamage       float64
rearRightTrunkDamage           float64
rearRightWheelDamage           float64
frontLeftBumperDamage          float64
frontLeftDoorDamage            float64
frontLeftFenderDamage          float64
frontLeftHeadlightDamage       float64
frontLeftHoodDamage            float64
frontLeftWindshieldDamage      float64
rearLeftBumperDamage           float64
rearLeftDoorDamage             float64
rearLeftQuarterPanelDamage     float64
rearLeftTaillightDamage        float64
rearLeftTrunkDamage            float64
rearLeftWheelDamage            float64
dtype: object

In [116]:
def get_damage_level(data_row, damage_considered_panels, detection_considered_panels):
    """
    Calculate the damage level for a particular row (representing a vehicle) using the specified columns.

    Parameters:
    - data_row: A pandas Series object representing a row of the DataFrame.
    - damage_considered_panels: List of panel columns to consider for damage calculations.
    - detection_considered_panels: List of panel columns to consider for critical detections.

    Returns:
    - Boolean: True if the damage level passes the thresholds, False otherwise.
    """
    image_damage_count = 0
    tot_panel_damage_count = 0
    detections_panel_count = 0
    fully_damaged_panel_count = 0

    # Iterate over all the damage-considered panels
    for panel in damage_considered_panels:
        panel_score = data_row[panel]
        if pd.notnull(panel_score):
            # Count the overall panel damage
            tot_panel_damage_count += (1 if panel_score > 0.1 else 0)
            # Fully damaged panels
            fully_damaged_panel_count += (1 if panel_score == 1 else 0)

    # Iterate over detection-considered panels
    for panel in detection_considered_panels:
        panel_score = data_row[panel]
        if pd.notnull(panel_score):
            detections_panel_count += (1 if panel_score == 1 else 0)

    # Image damage count is determined by non-zero damages
    image_damage_count = sum(
        1 for panel in damage_considered_panels if pd.notnull(data_row[panel]) and data_row[panel] > 0.1)

    # Apply the thresholds based on extracted logic
    if detections_panel_count >= 1:
        return False
    if fully_damaged_panel_count >= 3:
        return False
    if (image_damage_count > 2) or (tot_panel_damage_count > 4):
        return False
    if (image_damage_count == 1) and (tot_panel_damage_count >= 4):
        return False

    return True


# Define the lists for damage-considered and detection-considered panels
damage_considered_panels = [
    'frontRightBumperDamage', 'frontRightDoorDamage', 'frontRightFenderDamage', 'frontRightHeadlightDamage',
    'frontRightHoodDamage', 'frontRightWindshieldDamage', 'rearRightBumperDamage', 'rearRightDoorDamage',
    'rearRightQuarterPanelDamage', 'rearRightTaillightDamage', 'rearRightTrunkDamage', 'rearRightWheelDamage',
    'frontLeftBumperDamage', 'frontLeftDoorDamage', 'frontLeftFenderDamage', 'frontLeftHeadlightDamage',
    'frontLeftHoodDamage', 'frontLeftWindshieldDamage', 'rearLeftBumperDamage', 'rearLeftDoorDamage',
    'rearLeftQuarterPanelDamage', 'rearLeftTaillightDamage', 'rearLeftTrunkDamage', 'rearLeftWheelDamage'
]

detection_considered_panels = ['airbagsDeployedDetected', 'suspensionOrAxleDetected']

# Apply the function to the entire DataFrame
df['light_damage'] = df.apply(
    get_damage_level,
    axis=1,
    damage_considered_panels=damage_considered_panels,
    detection_considered_panels=detection_considered_panels
)

# Filter to keep only rows with an acceptable damage level
data = df[df['light_damage']]


In [120]:
data.shape

(1346, 58)

In [121]:
df

,lot_nbr,vin_nbr,yard_nbr,lot_year,make,lotModel,groupModel,ACV,plugACV,lot_cat_long_desc,...,rearLeftBumperDamage,rearLeftWindshieldDamage,rearLeftWheelDamage,rearLeftTrunkDamage,airbagsDeployedDetected,suspensionOrAxleDetected,dislocationDetected,flatTireDetected,lids_version,light_damage
0,57396094,5YJ3E1EA9NF186846,79,2022,TESL,MODEL 3,MODEL 3,26265.00,29825.0,V - VANS/MINIVANS,...,0.76655,0.0,0.009431,0.122461,0,0,1,0,"""3.0.2""",False
1,57414094,1N4AL3AP3JC212227,67,2018,NISS,ALTIMA 2.5,ALTIMA,14105.07,14975.0,A - AUTOMOBILE,...,0.00000,0.0,0.000000,0.000000,0,0,0,0,"""3.0.2""",False
2,57450684,4S3GTAD69J3750054,23,2018,SUBA,IMPREZA PR,IMPREZA,14458.82,16200.0,A - AUTOMOBILE,...,0.00000,0.0,0.000000,0.000000,0,0,0,0,"""3.0.2""",True
3,57406334,1GNEVJKW8KJ199639,345,2019,CHEV,TRAVERSE H,TRAVERSE,20841.57,32375.0,S - SUV'S,...,NaN,None,0.000000,NaN,0,0,0,0,"""3.0.2""",True
4,57397344,5XXGT4L30LG383718,369,2020,KIA,OPTIMA LX,OPTIMA,14569.56,17725.0,V - VANS/MINIVANS,...,0.00000,0.0,0.000000,0.000000,0,1,1,0,"""3.0.2""",False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2498,82515213,5N1AT2MV7LC749583,199,2020,NISS,ROGUE S,ROGUE,20609.20,20850.0,S - SUV'S,...,0.00000,0.0,0.000000,0.000000,0,0,1,0,"""3.0.2""",False
2499,82515693,1HGCV1F39LA128470,23,2020,HOND,ACCORD SPO,ACCORD,24599.10,22525.0,A - AUTOMOBILE,...,0.00000,0.0,0.000000,0.000000,0,0,0,1,"""3.0.2""",True
2500,82526584,KL7CJPSB3LB071183,23,2020,CHEV,TRAX 1LT,TRAX,13685.56,15950.0,S - SUV'S,...,0.00000,0.0,0.000000,0.000000,1,0,0,1,"""3.0.2""",False
2501,77051953,JM3KFBCM7J0333404,197,2018,MAZD,CX-5 TOURI,CX-5,21046.92,21175.0,S - SUV'S,...,0.00000,0.0,0.000000,0.000000,0,0,0,0,"""3.0.2""",False


In [64]:
# Define the column names where quotes need to be removed
columns_to_clean = ["frontRightImageUrl", "frontLeftImageUrl", "rearRightImageUrl", "rearLeftImageUrl"]
df=data.loc[data['totalAdvChrgsPercent']>0.25,]
# Remove double quotes from each of the specified columns
for col in columns_to_clean:
    df[col] = df[col].str.replace('"', '', regex=False)

# Verify the changes
df[columns_to_clean].head()


/var/folders/qs/jv302m6d68gd690wb7sp5c8h0000gq/T/ipykernel_25269/3785562295.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].str.replace('"', '', regex=False)


,frontRightImageUrl,frontLeftImageUrl,rearRightImageUrl,rearLeftImageUrl
0,https://c-static.copart.com/v1/AUTH_svc.pdoc00...,https://c-static.copart.com/v1/AUTH_svc.pdoc00...,https://c-static.copart.com/v1/AUTH_svc.pdoc00...,https://c-static.copart.com/v1/AUTH_svc.pdoc00...
2,https://c-static.copart.com/v1/AUTH_svc.pdoc00...,https://c-static.copart.com/v1/AUTH_svc.pdoc00...,https://c-static.copart.com/v1/AUTH_svc.pdoc00...,https://c-static.copart.com/v1/AUTH_svc.pdoc00...
3,https://c-static.copart.com/v1/AUTH_svc.pdoc00...,https://c-static.copart.com/v1/AUTH_svc.pdoc00...,https://c-static.copart.com/v1/AUTH_svc.pdoc00...,https://c-static.copart.com/v1/AUTH_svc.pdoc00...
4,https://c-static.copart.com/v1/AUTH_svc.pdoc00...,https://c-static.copart.com/v1/AUTH_svc.pdoc00...,https://c-static.copart.com/v1/AUTH_svc.pdoc00...,https://c-static.copart.com/v1/AUTH_svc.pdoc00...
9,https://c-static.copart.com/v1/AUTH_svc.pdoc00...,https://c-static.copart.com/v1/AUTH_svc.pdoc00...,https://c-static.copart.com/v1/AUTH_svc.pdoc00...,https://c-static.copart.com/v1/AUTH_svc.pdoc00...


In [54]:
main_category_subsets.head()

AttributeError: 'dict' object has no attribute 'head'

In [70]:

data.modelYear.describe()

count      10000.0
mean     2013.6061
std        5.61903
min         1972.0
25%         2011.0
50%         2014.0
75%         2017.0
max         2024.0
Name: modelYear, dtype: Float64

In [87]:
# Define the damage types to filter
damage_types = ['FR', 'RR', 'SD', 'AO', 'RO']

# Filter the DataFrame to include only rows with the specified damage types
filtered_data = dfx[dfx['damage_type_cd'].isin(damage_types)]

# Verify the filtered data
filtered_data.head()


,lot_nbr,vin_nbr,yard_nbr,lot_year,make,lotModel,groupModel,ACV,plugACV,lot_cat_long_desc,...,rearLeftBumperDamage,rearLeftWindshieldDamage,rearLeftWheelDamage,rearLeftTrunkDamage,airbagsDeployedDetected,suspensionOrAxleDetected,dislocationDetected,flatTireDetected,lids_version,teardown_flag
0,40267394,JF1VBAA62P9813294,7,2023,SUBA,WRX,WRX,27320.0,27875.0,A - AUTOMOBILE,...,None,None,0.0,None,0,0,0,0,"""4.0.0""",No
1,40207774,JA4AZ3A39LZ018351,116,2020,MITS,OUTLANDER,OUTLANDER,18125.0,18575.0,S - SUV'S,...,0.0,0.0,0.0,None,0,0,0,0,"""4.0.0""",No
2,40237174,YV4A22RL4J1066664,24,2018,VOLV,XC60 T6 IN,XC60,26696.0,26075.0,A - AUTOMOBILE,...,0.0,0.0,0.0,0.0,1,0,0,1,"""4.0.0""",Yes
3,39430294,5FPYK3F88NB006897,114,2022,HOND,RIDGELINE,RIDGELINE,32103.0,38175.0,P - PICKUP'S,...,0.0,None,0.0,0.0,0,0,0,0,"""4.0.0""",No
4,39295194,KNDPNCAC3J7428616,27,2018,KIA,SPORTAGE E,SPORTAGE,19000.0,17475.0,V - VANS/MINIVANS,...,0.0,0.0,0.0,0.0,1,1,0,1,"""4.0.0""",No


In [108]:
data.shape

(4897, 58)

In [118]:
# Define the column names where quotes need to be removed
data['teardown_flag'] = np.where(
    (data['frontRightBumperDamage'].isnull() & data['frontLeftBumperDamage'].isnull() &
     (data['frontRightHoodDamage'] <= 0.2) & (data['frontLeftHoodDamage'] <= 0.2)) |
    (data['frontRightFenderDamage'].isnull() & (data['frontRightHoodDamage'] < 0.2)) |
    (data['frontLeftFenderDamage'].isnull() & (data['frontLeftHoodDamage'] <= 0.2)),
    'Yes', 'No'
)

columns_to_clean = ["frontRightImageUrl", "frontLeftImageUrl", "rearRightImageUrl", "rearLeftImageUrl"]
dfx=data.loc[(data['totalAdvChrgsPercent']>0.25) & (data['lot_year']>2015) & (data['teardown_flag']=='No'),]
# Remove double quotes from each of the specified columns
for col in columns_to_clean:
    dfx[col] = dfx[col].str.replace('"', '', regex=False)

# Verify the changes
dfx[columns_to_clean].head()

/var/folders/qs/jv302m6d68gd690wb7sp5c8h0000gq/T/ipykernel_25269/3449490723.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['teardown_flag'] = np.where(
/var/folders/qs/jv302m6d68gd690wb7sp5c8h0000gq/T/ipykernel_25269/3449490723.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfx[col] = dfx[col].str.replace('"', '', regex=False)


,frontRightImageUrl,frontLeftImageUrl,rearRightImageUrl,rearLeftImageUrl
3,https://c-static.copart.com/v1/AUTH_svc.pdoc00...,https://c-static.copart.com/v1/AUTH_svc.pdoc00...,https://c-static.copart.com/v1/AUTH_svc.pdoc00...,https://c-static.copart.com/v1/AUTH_svc.pdoc00...
5,https://c-static.copart.com/v1/AUTH_svc.pdoc00...,https://c-static.copart.com/v1/AUTH_svc.pdoc00...,https://c-static.copart.com/v1/AUTH_svc.pdoc00...,https://c-static.copart.com/v1/AUTH_svc.pdoc00...
12,https://c-static.copart.com/v1/AUTH_svc.pdoc00...,https://c-static.copart.com/v1/AUTH_svc.pdoc00...,https://c-static.copart.com/v1/AUTH_svc.pdoc00...,https://c-static.copart.com/v1/AUTH_svc.pdoc00...
13,https://c-static.copart.com/v1/AUTH_svc.pdoc00...,https://c-static.copart.com/v1/AUTH_svc.pdoc00...,https://c-static.copart.com/v1/AUTH_svc.pdoc00...,https://c-static.copart.com/v1/AUTH_svc.pdoc00...
15,https://c-static.copart.com/v1/AUTH_svc.pdoc00...,https://c-static.copart.com/v1/AUTH_svc.pdoc00...,https://c-static.copart.com/v1/AUTH_svc.pdoc00...,https://c-static.copart.com/v1/AUTH_svc.pdoc00...


In [107]:
# Define the column names where quotes need to be removed
data['teardown_flag'] = np.where(
    (data['frontRightBumperDamage'].isnull() & data['frontLeftBumperDamage'].isnull() &
     (data['frontRightHoodDamage'] <= 0.2) & (data['frontLeftHoodDamage'] <= 0.2)) |
    (data['frontRightFenderDamage'].isnull() & (data['frontRightHoodDamage'] < 0.2)) |
    (data['frontLeftFenderDamage'].isnull() & (data['frontLeftHoodDamage'] <= 0.2)),
    'Yes', 'No'
)

columns_to_clean = ["frontRightImageUrl", "frontLeftImageUrl", "rearRightImageUrl", "rearLeftImageUrl"]
dfx=data.loc[(data['totalAdvChrgsPercent']>0.25) & (data['lot_year']>2015) & (data['teardown_flag']=='No'),]
# Remove double quotes from each of the specified columns
for col in columns_to_clean:
    dfx[col] = dfx[col].str.replace('"', '', regex=False)

# Verify the changes
dfx[columns_to_clean].head()


main_categories = ['A - AUTOMOBILE', 'S - SUV\'S', 'P - PICKUP\'S', 'V - VANS/MINIVANS']

# Create a dictionary to store subsets
main_category_subsets = {}

for category in main_categories:
    main_category_subsets[category] = df[df['lot_cat_long_desc'] == category].sample(n=500,
                                                                                                    random_state=42)

# Step 2: Combine main category subsets into a single DataFrame
subset_main_categories = pd.concat(main_category_subsets.values())

# Step

/var/folders/qs/jv302m6d68gd690wb7sp5c8h0000gq/T/ipykernel_25269/2944201360.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['teardown_flag'] = np.where(
/var/folders/qs/jv302m6d68gd690wb7sp5c8h0000gq/T/ipykernel_25269/2944201360.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].str.replace('"', '', regex=False)


,frontRightImageUrl,frontLeftImageUrl,rearRightImageUrl,rearLeftImageUrl
1,https://c-static.copart.com/v1/AUTH_svc.pdoc00...,https://c-static.copart.com/v1/AUTH_svc.pdoc00...,https://c-static.copart.com/v1/AUTH_svc.pdoc00...,https://c-static.copart.com/v1/AUTH_svc.pdoc00...
15,https://c-static.copart.com/v1/AUTH_svc.pdoc00...,https://c-static.copart.com/v1/AUTH_svc.pdoc00...,https://c-static.copart.com/v1/AUTH_svc.pdoc00...,https://c-static.copart.com/v1/AUTH_svc.pdoc00...
17,https://c-static.copart.com/v1/AUTH_svc.pdoc00...,https://c-static.copart.com/v1/AUTH_svc.pdoc00...,https://c-static.copart.com/v1/AUTH_svc.pdoc00...,https://c-static.copart.com/v1/AUTH_svc.pdoc00...
24,https://c-static.copart.com/v1/AUTH_svc.pdoc00...,https://c-static.copart.com/v1/AUTH_svc.pdoc00...,https://c-static.copart.com/v1/AUTH_svc.pdoc00...,https://c-static.copart.com/v1/AUTH_svc.pdoc00...
26,https://c-static.copart.com/v1/AUTH_svc.pdoc00...,https://c-static.copart.com/v1/AUTH_svc.pdoc00...,https://c-static.copart.com/v1/AUTH_svc.pdoc00...,https://c-static.copart.com/v1/AUTH_svc.pdoc00...


ValueError: Cannot take a larger sample than population when 'replace=False'

In [84]:
subset_main_categories.head()

,lot_nbr,vin_nbr,yard_nbr,lot_year,make,lotModel,groupModel,ACV,plugACV,lot_cat_long_desc,...,rearLeftBumperDamage,rearLeftWindshieldDamage,rearLeftWheelDamage,rearLeftTrunkDamage,airbagsDeployedDetected,suspensionOrAxleDetected,dislocationDetected,flatTireDetected,lids_version,teardown_flag
4686,81655913,3N1AB8CV8LY219449,193,2020,NISS,SENTRA SV,SENTRA,17043.0,18125.0,A - AUTOMOBILE,...,0.0,0.0,0.0,0.0,1,1,0,0,"""4.0.0""",No
612,80474643,JF2GTAEC9M8320229,24,2021,SUBA,CROSSTREK,CROSSTREK,22105.0,24875.0,A - AUTOMOBILE,...,None,0.8479280688,0.0,None,0,0,0,0,"""4.0.0""",No
1844,77043493,3N1AB7AP9KY345770,56,2019,NISS,SENTRA S,SENTRA,10509.0,13525.0,A - AUTOMOBILE,...,0.0,0.0,0.0,0.0,0,1,0,1,"""4.0.0""",No
824,80293173,3MW5R1J05M8C07458,98,2021,BMW,330I,3 SERIES,28771.2,31775.0,A - AUTOMOBILE,...,0.0,0.0,0.0,0.0,0,0,0,0,"""4.0.0""",No
8849,78352363,2G1125S33J9147870,130,2018,CHEV,IMPALA PRE,IMPALA,14810.0,18475.0,A - AUTOMOBILE,...,0.0,0.0,0.0,0.0,0,0,0,0,"""4.0.0""",No


In [119]:
dfx.to_excel('subset_main_categories3.xlsx')

In [59]:
import os

In [60]:
os.getcwd()

'/Users/rokokkula/Library/CloudStorage/OneDrive-Copart,Inc/Mac Backup Feb 5, 2025/Documents mac/scientificProject'